In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

import torch
import torch.nn as nn

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *

In [2]:
DATA_DIR = "/Volumes/SSD970/"
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")
train_df = pd.read_csv(os.path.join(DATA_DIR, "train_df_with_slices.csv"))

In [3]:
patient_id = "1.2.826.0.1.3680043.780"

In [4]:
imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses = read_patient_dcm(train_df, TRAIN_IMAGE_DIR,patient_id)
imgs.shape

(195, 512, 512)

In [5]:
height = image_positions[0, 2] - image_positions[1, 2]
pixel_spacing = pixel_spacings[0, 0]
aspect = np.round(height / pixel_spacing)
aspect

3.0

In [6]:
img3d = np.repeat(imgs, aspect, axis=0)
img3d.shape

(585, 512, 512)

In [7]:
img3d[:, 250, 250]

array([-3.100e+01, -3.100e+01, -3.100e+01, -5.000e+01, -5.000e+01,
       -5.000e+01,  9.000e+00,  9.000e+00,  9.000e+00, -2.100e+01,
       -2.100e+01, -2.100e+01, -2.300e+01, -2.300e+01, -2.300e+01,
       -9.500e+01, -9.500e+01, -9.500e+01, -1.100e+01, -1.100e+01,
       -1.100e+01,  4.000e+01,  4.000e+01,  4.000e+01, -2.800e+01,
       -2.800e+01, -2.800e+01,  2.200e+01,  2.200e+01,  2.200e+01,
        1.100e+01,  1.100e+01,  1.100e+01,  5.000e+00,  5.000e+00,
        5.000e+00,  3.600e+01,  3.600e+01,  3.600e+01,  6.400e+01,
        6.400e+01,  6.400e+01,  3.100e+01,  3.100e+01,  3.100e+01,
        4.000e+01,  4.000e+01,  4.000e+01,  5.100e+01,  5.100e+01,
        5.100e+01,  4.400e+01,  4.400e+01,  4.400e+01,  2.800e+01,
        2.800e+01,  2.800e+01, -1.500e+01, -1.500e+01, -1.500e+01,
       -1.600e+01, -1.600e+01, -1.600e+01,  5.000e+00,  5.000e+00,
        5.000e+00, -1.000e+01, -1.000e+01, -1.000e+01, -9.200e+01,
       -9.200e+01, -9.200e+01, -1.090e+02, -1.090e+02, -1.090e

In [8]:
img3d[150, :, :].max()

2125.0

In [9]:
img3d[150, :, :].min()

-1024.0

Normalization

In [10]:
img3d = img3d / 2048

In [11]:
input = torch.tensor(img3d, dtype=torch.float).unsqueeze(0)
input.shape # C, D, H, W

torch.Size([1, 585, 512, 512])